In [ ]:
"""

Date : August 1, 2019

Author : Heather Landry Drexler

This script will develop the datasets necessary for making percent spliced by 
distance transcribed plots from nano-COP data. These files serve as input for the
scripts for Figures 2, 3 and S5.
                                            
"""


In [11]:
import numpy as np
import pandas as pd
import re
import math
import pybedtools
from pybedtools import BedTool


In [12]:
def get_discarded_features_wiRNAPET_wiNoPolyA_bedtool(annotation_df, polyA_window, ss_window_upstream, ss_window_downstream, RNAPET_df, RNAPET_window, nopolyA_bamFile):

    # make a set for all 3'SS coordinates
    features = []
    
    # loop through a file with intron coordinates
    for i in range(0,len(annotation_df)):
        feature = annotation_df['feature'].iloc[i].split("_")[0]    # feature
         
        # check if feature is a gene and record region around polyA site
        if (feature == 'gene'):
            chrom = 'chr'+str(annotation_df['chrom'].iloc[i])       # chromosome
            start = int(annotation_df['start'].iloc[i])             # start coordinate of intron (last base of exon)
            end = int(annotation_df['end'].iloc[i])                 # end coordinate of intron (last base of intron)
            gene = annotation_df['gene'].iloc[i]                    # gene name
            strand = annotation_df['strand'].iloc[i]                # strand of gene with intron
            name = annotation_df['feature'].iloc[i]                 # get feature and count for output file
   
            if (strand=='+'):
                polyA_start = end - polyA_window
                polyA_end = end + polyA_window
                
            if (strand=='-'):
                polyA_start = start - polyA_window
                polyA_end = start + polyA_window

            if (polyA_window > 0):
                features.append([chrom,polyA_start,polyA_end,gene,'polyA',strand])
            
        # check if feature is an intron and record region around 5'SS site   
        if (feature == 'intron'):
            chrom = 'chr'+str(annotation_df['chrom'].iloc[i])       # chromosome
            start = int(annotation_df['start'].iloc[i])             # start coordinate of intron (last base of exon)
            end = int(annotation_df['end'].iloc[i])                 # end coordinate of intron (last base of intron)
            gene = annotation_df['gene'].iloc[i]                    # gene name
            strand = annotation_df['strand'].iloc[i]                # strand of gene with intron
            name = annotation_df['feature'].iloc[i]                 # get feature and count for output file
   
            # get 5' splice site positions for introns on each strand
            if (strand=='+'):
                ss5_start = int(start)-ss_window_upstream
                ss5_end = int(start)+ss_window_downstream
            if (strand=='-'):
                ss5_start = int(end)-ss_window_downstream
                ss5_end = int(end)+ss_window_upstream

            if (ss5_start > 0 and ss5_end > ss5_start):
                features.append([chrom,ss5_start,ss5_end,gene,name+"_SS",strand])

    # record regions around 3' end RNA-PET coordinates from ENCODE data
    for i in range(0,len(RNAPET_df)):

        # get features of line in RNA-PET bed file
        chrom = str(RNAPET_df.iloc[i]['chrom'])
        start = RNAPET_df.iloc[i]['start']
        end = RNAPET_df.iloc[i]['end']
        ID = RNAPET_df.iloc[i]['ID']
        score = RNAPET_df.iloc[i]['score'].astype(str)
        strand = RNAPET_df.iloc[i]['strand']
        size = RNAPET_df.iloc[i]['size'].split(',')
        loc = RNAPET_df.iloc[i]['loc'].split(',')

        # process file based on read strand
        # make a new bed file with only information about the polyA site

        if (strand == "+"):
            polyA_start = end - int(size[1]) + 1 - RNAPET_window
            polyA_end = end + RNAPET_window

        if (strand == "-"): 
            polyA_start = start + 1 - RNAPET_window
            polyA_end = start + int(size[0]) + RNAPET_window

        if (polyA_start > 0):
            features.append([chrom,polyA_start,polyA_end,ID,'polyA',strand])
        
    nopolyA_df = nopolyA_bamFile.bam_to_bed().to_dataframe()
    for i in range(len(nopolyA_df)):

        chrom = "chr"+str(nopolyA_df.iloc[i]['chrom'])
        read_start = nopolyA_df.iloc[i]['start']
        read_end = nopolyA_df.iloc[i]['end']
        read_name = nopolyA_df.iloc[i]['name']
        strand = nopolyA_df.iloc[i]['strand']
    
        if strand=="+":

            polyA_start = int(read_end)-50
            polyA_end = int(read_end)+50

        if strand=="-":

            polyA_start = int(read_start)-50
            polyA_end = int(read_start)+50

        if (polyA_start > 0):
            features.append([chrom,polyA_start,polyA_end,read_name,'polyA',strand])    

    features_bedtool = BedTool(features)
    return features_bedtool


def get_discarded_features_woRNAPET_bedtool(annotation_df, polyA_window, ss_window_upstream, ss_window_downstream):

    # make a set for all 3'SS coordinates
    features = []

    # loop through a file with intron coordinates
    # check if feature is an exon
    for i in range(0,len(annotation_df)):
        feature = annotation_df['feature'].iloc[i].split("_")[0]    # feature
 
        if (feature == 'gene'):
            chrom = 'chr'+str(annotation_df['chrom'].iloc[i])       # chromosome
            start = int(annotation_df['start'].iloc[i])             # start coordinate of intron (last base of exon)
            end = int(annotation_df['end'].iloc[i])                 # end coordinate of intron (last base of intron)
            gene = annotation_df['gene'].iloc[i]                    # gene name
            strand = annotation_df['strand'].iloc[i]                # strand of gene with intron
            name = annotation_df['feature'].iloc[i]                 # get feature and count for output file
   
            if (strand=='+'):
                polyA_start = end - polyA_window
                polyA_end = end + polyA_window
                
            if (strand=='-'):
                polyA_start = start - polyA_window
                polyA_end = start + polyA_window

            if (polyA_window > 0):
                features.append([chrom,polyA_start,polyA_end,gene,'polyA',strand])
            
   
        if (feature == 'intron'):
            chrom = 'chr'+str(annotation_df['chrom'].iloc[i])       # chromosome
            start = int(annotation_df['start'].iloc[i])             # start coordinate of intron (last base of exon)
            end = int(annotation_df['end'].iloc[i])                 # end coordinate of intron (last base of intron)
            gene = annotation_df['gene'].iloc[i]                    # gene name
            strand = annotation_df['strand'].iloc[i]                # strand of gene with intron
            name = annotation_df['feature'].iloc[i]                 # get feature and count for output file
   
            # get 5' splice site positions for introns on each strand
            if (strand=='+'):
                ss5_start = int(start)-ss_window_upstream
                ss5_end = int(start)+ss_window_downstream
            if (strand=='-'):
                ss5_start = int(end)-ss_window_downstream
                ss5_end = int(end)+ss_window_upstream

            if (ss5_start > 0 and ss5_end > ss5_start):
                features.append([chrom,ss5_start,ss5_end,gene,name+"_SS",strand])

    features_bedtool = BedTool(features)
    return features_bedtool


# make a bed tool with coordinates of read 3' ends
def get_read_end_bedtool(bamFile):

    bedFile = bamFile.bam_to_bed()
    bedFile_df = bedFile.to_dataframe(names=['chrom', 'start', 'end', 'name', 'score', 'strand'], \
                               dtype={"chrom": str, "start": int, "end": int, "name": str, \
                                      "score": int, "strand": str}) # convert to a dataframe       
    read_end = []
        
    for i in range(0,len(bedFile_df)):

        chrom = 'chr'+str(bedFile_df['chrom'].iloc[i])
        start = bedFile_df['start'].iloc[i]
        end = bedFile_df['end'].iloc[i]
        read = bedFile_df['name'].iloc[i]
        score = bedFile_df['score'].iloc[i]
        strand = bedFile_df['strand'].iloc[i]

        if (strand == "-"):
            pos_1 = start
            pos_2 = start + 1

        if (strand == "+"):
            pos_1 = end - 1
            pos_2 = end

        read_end.append([chrom,pos_1,pos_2,read,score,strand])

    read_end_bedtool = BedTool(read_end)
    return read_end_bedtool


# intersect 3' ends with discarded features bed file 
# to get reads that will be removed from the analysis
def get_discarded_reads(bamFile, features):

    # get read ends and turn into a bedtool for intersecting 
    read_ends_bedtool = get_read_end_bedtool(bamFile)

    # intersect read ends with genome features
    intersect = read_ends_bedtool.intersect(features, wo=True, s=True) # intersect reads from bam file with 3' splice site coordinates, ensure strandedness
    intersect_df = intersect.to_dataframe(names=['chrom_read', 'start_read', 'end_read', 'name_read', 'qual_read', \
                                           'strand_read', 'chr_feature', 'start_feature', \
                                           'end_feature', 'name_gene', 'name_feature', 'strand_feature', 'count'], \
                               dtype={"chrom_read": str, "start_read": int, "end_read": int, \
                                     "name_read": str, "qual_read": int, "strand_read": str, \
                                    "chr_feature": str, "start_feature": int, "end_feature": int, "name_gene": str, \
                                     "name_feature": str,"strand_feature": str, "count": int}) # convert to a dataframe

    unique_read_names = set(intersect_df['name_read'])

    return unique_read_names


# make a bed tool with intron coordinates
def get_introns_bedtool(annotation_df, window):

    # make a set for all 3'SS coordinates
    introns = []

    # loop through a file with intron coordinates
    # check if feature is an exon
    for i in range(0,len(annotation_df)):
        feature = annotation_df['feature'].iloc[i].split("_")[0]    # feature
 
        if (feature == 'intron'):
            chrom = annotation_df['chrom'].iloc[i]                  # chromosome
            start = int(annotation_df['start'].iloc[i])             # start coordinate of intron (last base of exon)
            end = int(annotation_df['end'].iloc[i])                 # end coordinate of intron (last base of intron)
            gene = annotation_df['gene'].iloc[i]                    # gene name
            strand = annotation_df['strand'].iloc[i]                # strand of gene with intron
            name = annotation_df['feature'].iloc[i]                 # get feature and count for output file
   
            if (end-start > 2*window):
                start = start + window
                end = end - window 

            introns.append([chrom,start,end,gene,name,strand])
    
    introns_bedtool = BedTool(introns)
    return introns_bedtool

# function to get a bedtool file with splice site info from hg38 intron coordinate bed file
def human_spliceSites(intronFile):
    # make a set for all 3'SS coordinates
    introns = []

    # loop through a file with intron coordinates
    # record features about the introns
    for line in intronFile:
        chrom = str(line.split('\t')[0])      # chromosome
        start = line.split('\t')[1]                 # start coordinate of intron (last base of exon)
        end = line.split('\t')[2]                   # end coordinate of intron (last base of intron)
        name = line.split('\t')[3]                  # gene name and polyA site for that gene (multiples are split with "_")
        strand = line.split('\t')[4][0]             # strand of gene with intron

        # get 3' SS positions for introns plus one base
        if strand=='+':
            pos1 = int(end)
            pos2 = int(end)+1
            pos5prime = int(start)+1
        if strand=='-':
            pos1 = int(start)
            pos2 = int(start)+1
            pos5prime = int(end)

        # make a key that will represent intron coordinates
        introns.append([str(chrom),str(pos1),str(pos2),str(name),str(pos5prime),str(strand)])

    spliceSites = BedTool(introns)
    intronFile.close()
    return spliceSites


# function to get a bedtool file with splice site info from dm6 intron coordinate bed file
def drosophila_spliceSites(intronFile):
    # make a set for all 3'SS coordinates
    introns = []

    # loop through a file with intron coordinates
    # record features about the introns
    for line in intronFile:
        chrom = 'chr'+str(line.split('\t')[0])      # chromosome
        start = line.split('\t')[1]                 # start coordinate of intron (last base of exon)
        end = line.split('\t')[2]                   # end coordinate of intron (last base of intron)
        name = line.split('\t')[3]                  # gene name and polyA site for that gene (multiples are split with "_")
        strand = line.split('\t')[4][0]             # strand of gene with intron

        # get 3' SS positions for introns plus one base
        if strand=='+':
            pos1 = int(end)
            pos2 = int(end)+1
            pos5prime = int(start)+1
        if strand=='-':
            pos1 = int(start)
            pos2 = int(start)+1
            pos5prime = int(end)

        # make a key that will represent intron coordinates
        introns.append([str(chrom),str(pos1),str(pos2),str(name),str(pos5prime),str(strand)])

    spliceSites = BedTool(introns)
    intronFile.close()
    return spliceSites


# function to create a dataframe with reads that span 3'SS positions
def get_spliceSite_df(spliceSites, bamFile):
    # get reads that span 3' splice sites and convert to a dataframe
    bedFile = bamFile.bam_to_bed(cigar=True, tag='NM') # convert bam file to bed file, keep cigar string and NM (edit distance) tag
    intersect = bedFile.intersect(spliceSites, wo=True, s=True) # intersect reads from bam file with 3' splice site coordinates, ensure strandedness
    df = intersect.to_dataframe(names=['chrom', 'start_aln', 'end_aln', 'name_aln', 'qual_aln', \
                                           'strand_aln', 'cigar_aln', 'chr_3SS', 'start_3SS', \
                                           'end_3SS', 'name_gene_polyA', 'pos_5SS', 'strand_gene', 'count'], \
                               dtype={"chrom": str, "start_aln": int, "end_aln": int, \
                                     "name_aln": str, "qual_aln": int, "strand_aln": str, \
                                     "cigar_aln": str, "chr_3SS": str, "start_3SS": int, \
                                     "end_3SS": int, "name_gene_polyA": str, \
                                     "pos_5SS": int,"strand_gene": str, "count": int}) # convert to a dataframe
    return df


def get_MinION_spliceCalls(df, min_overlap):
    
    # prepare a list for splice calls
    spliceCalls = []

    # set variables for parsing the cigar string
    pattern = re.compile('([MIDNSHPX=])')
    Consumes_Query = ["M", "I", "S", "=", "X"]
    Consumes_Reference = ["M", "D", "N", "=", "X"]    

    # loop through all reads that span splice sites
    for i in range(0,df.shape[0]):
        if df.strand_gene[i] == "-":
            align_3p_end = df.start_aln[i] # record 3' end of read for - strand genes
            align_5p_end = df.end_aln[i] # record 5' end of read for - strand genes
            pos_3SS = df.end_3SS[i] # record 3'SS position for - strand genes
            pos_5SS = df.pos_5SS[i] # record 5'SS position for - strand genes
            intron_start = pos_3SS # get position for the start of intron coordinate on negative strand
            intron_end = pos_5SS # get position for the end of intron coordinate on negative strand

        if df.strand_gene[i] == "+":
            align_3p_end = df.end_aln[i] # record 3' end of read for + strand genes
            align_5p_end = df.start_aln[i] # record 5' end of read for + strand genes
            pos_3SS = df.start_3SS[i] # record 3'SS position for + strand genes 
            pos_5SS = df.pos_5SS[i] # record 5'SS position for - strand genes 
            intron_start = pos_5SS # get position for the start of intron coordinate on positive strand
            intron_end = pos_3SS # get position for the end of intron coordinate on positive strand
            
        # calculate distance between 3'SS and 3'end of read 
        dist = abs(align_3p_end - pos_3SS) #*** double check this!!!

        # parse cigar string into a list of tuples for easy parsing
        Sep_Values = pattern.split(df.cigar_aln[i])[:-1]
        CigarPairs = list((Sep_Values[n:n+2] for n in range(0, len(Sep_Values), 2)))  

        # get the 3' softclip length
        if df.strand_aln[i]=="+":        
            last=len(CigarPairs)
            if(CigarPairs[last-1][1]=='S'):
                clip=CigarPairs[last-1][0]
            elif(CigarPairs[last-1][1]=='H'):
                clip=CigarPairs[last-1][0]
            elif(CigarPairs[last-1][1]!='S' or CigarPairs[last-1][1]!='H'):
                clip=0

        if df.strand_aln[i]=="-":
            if(CigarPairs[0][1]=='S'):
                clip=CigarPairs[0][0]
            elif(CigarPairs[0][1]=='H'):
                clip=CigarPairs[0][0]
            elif(CigarPairs[0][1]!='S' or CigarPairs[0][1]!='H'):
                clip=0

        # set up variables for measuring the length of cigar string operators
        CigarOp_counts = {'M': 0, 'I': 0, 'D': 0, 'N': 0, 'S': 0, 'H': 0, 'P': 0, '=': 0, 'X': 0}
        start_counts = {'M': 0, 'I': 0, 'D': 0, 'N': 0, 'S': 0, 'H': 0, 'P': 0, '=': 0, 'X': 0}
        end_counts = {'M': 0, 'I': 0, 'D': 0, 'N': 0, 'S': 0, 'H': 0, 'P': 0, '=': 0, 'X': 0}
        intron_counts = {'M': 0, 'I': 0, 'D': 0, 'N': 0, 'S': 0, 'H': 0, 'P': 0, '=': 0, 'X': 0}
        currentloc = int(df.start_aln[i]) 

        # go through list of cigar strings and grab splicing information
        for cigar_Entry in CigarPairs:

            op_Length = int(cigar_Entry[0]) # get length of cigar operator
            cigarOp = cigar_Entry[1] # get type of cigar operator  
            CigarOp_counts[cigarOp] += op_Length # add the cigar operator length to the counts dictionary
            cigarOp_start=currentloc # get the starting coordinate of the cigar operator

            if (cigarOp in Consumes_Reference):
                currentloc=currentloc+op_Length # add the cigar operator length to the current location coordinate 

            cigarOp_end=currentloc # get the ending coordinate of the cigar operator

            # gather information if the portion of the cigar string spans the designated 5' splice site
            if (cigarOp_start<intron_start-min_overlap and cigarOp_end>=intron_start-min_overlap):
                if (cigarOp_end>=intron_start+min_overlap):
                    count=min_overlap*2
                else:
                    count=cigarOp_end-(intron_start-min_overlap)+1
                start_counts[cigarOp] += count # add the cigar operator length to the counts dictionary

            elif (cigarOp_start>=intron_start-min_overlap and cigarOp_end<intron_start+min_overlap):
                count=op_Length
                start_counts[cigarOp] += count # add the cigar operator length to the counts dictionary       

            elif (cigarOp_start<intron_start+min_overlap and cigarOp_end>=intron_start+min_overlap):
                if (cigarOp_start<=intron_start-min_overlap):
                    count=min_overlap*2
                else:
                    count=(intron_start+min_overlap)-cigarOp_start-1
                start_counts[cigarOp] += count # add the cigar operator length to the counts dictionary 

            # gather information if the portion of the cigar string is within the intron
            if (cigarOp_start<intron_start and cigarOp_end>=intron_start):
                if (cigarOp_end>=intron_end):
                    count=intron_end-intron_start
                else:
                    count=cigarOp_end-intron_start
                intron_counts[cigarOp] += count # add the cigar operator length to the counts dictionary

            elif (cigarOp_start>=intron_start and cigarOp_end<intron_end):
                count=op_Length
                intron_counts[cigarOp] += count # add the cigar operator length to the counts dictionary

            elif (cigarOp_start<intron_end and cigarOp_end>=intron_end):
                if (cigarOp_start<=intron_start):
                    count=intron_end-intron_start
                else:
                    count=intron_end-cigarOp_start
                intron_counts[cigarOp] += count # add the cigar operator length to the counts dictionary 

            # gather information if the portion of the cigar string spans the designated 3' splice site
            if (cigarOp_start<intron_end-min_overlap and cigarOp_end>=intron_end-min_overlap):
                if (cigarOp_end>=intron_end+min_overlap):
                    count=min_overlap*2
                else:
                    count=cigarOp_end-(intron_end-min_overlap)
                end_counts[cigarOp] += count # add the cigar operator length to the counts dictionary

            elif (cigarOp_start>=intron_end-min_overlap and cigarOp_end<intron_end+min_overlap):
                count=op_Length
                end_counts[cigarOp] += count # add the cigar operator length to the counts dictionary

            elif (cigarOp_start<intron_end+min_overlap and cigarOp_end>=intron_end+min_overlap):
                if (cigarOp_start<=intron_end-min_overlap):
                    count=min_overlap*2
                else:
                    count=(intron_end+min_overlap)-cigarOp_start
                end_counts[cigarOp] += count # add the cigar operator length to the counts dictionary 

        # assign strandedness to determine counts around 5'SS and 3'SS
        if(df.strand_gene[i]=='+'):
            around5SS_counts = start_counts
            around3SS_counts = end_counts

        elif(df.strand_gene[i]=="-"):
            around5SS_counts = end_counts
            around3SS_counts = start_counts

        # annotate splicing status based on CIGAR string information around splice sites
        if(around3SS_counts['N']==0 and around3SS_counts['M']>min_overlap/2):
            splice='NO'
        elif(around3SS_counts['N']>0 and around3SS_counts['N']<min_overlap*2):
            if(around5SS_counts['N']>0 and around5SS_counts['N']<min_overlap*2):
                splice='YES'
            else:
                splice='UNDETERMINED'
        else:
            splice='UNDETERMINED'

        # annotate splicing status based on CIGAR string information within the intron 
        if (splice == 'YES'):
            if (float(intron_end-intron_start) > 0.0):
                ratio = float(intron_counts['N'])/float(intron_end-intron_start)
                difference = abs(intron_counts['N']-(intron_end-intron_start))
                # if read is spliced, between 90-100% of the intron has to be spliced 
                # and no more than 100 nucleotides within the intron can be matching the intron sequence
                if( ratio < 0.9 or ratio > 1.1 or difference > 100):
                    splice='UNDETERMINED'
            if (float(intron_end-intron_start) == 0.0):
                splice='UNDETERMINED'

        if (splice == 'NO'):
            if (float(intron_end-intron_start) > 0.0):
                intronLength = intron_end-intron_start
                difference = abs(intron_counts['M']+intron_counts['D']+intron_counts['S']-intronLength)
                ratio = float(intron_counts['M'])/(float(intron_counts['M'])+float(intron_counts['N'])+float(intron_counts['D'])+1)
                # if read is unspliced, at least 70% of the read has to match (CIGAR=M) the intron sequence
                # and at least 10 nucleotides must match (CIGAR=M) within the intron sequence
                if(intron_counts['M'] < 10 or ratio < 0.7):
                    splice='UNDETERMINED'
            if (float(intron_end-intron_start) == 0.0):
                splice='UNDETERMINED'
        
        
        # get information on match percentage / error length 
        # this will be used as a quality control cutoff if necessary
        read_length = CigarOp_counts['M']+CigarOp_counts['I']
        error_rate = float(df.qual_aln[i])/float(read_length)

        spliceCalls.append([df.name_aln[i],df.chrom[i],str(intron_start),str(intron_end),str(align_5p_end),str(align_3p_end),str(read_length),df.strand_gene[i],str(error_rate),str(clip),str(dist),splice,df.name_gene_polyA[i]])
    
    spliceCalls_df = pd.DataFrame(spliceCalls)
    spliceCalls_df.columns = ["read_name","chrom","intron_start","intron_end","read_start","read_end","read_length","strand","error_rate","end_clippling","dist_from_3SS","splice_status","gene_name_polyA"]
   
    return spliceCalls_df


def get_MinION_spliceCalls_noPolyA(gene_name_polyA, strand, read_end, polyA_dist):

    # delimeter to identify if read has a 3' end near a polyA site
    near_polyA = 'NO'

    # count the number of polyA sites associated with a gene
    polyA_count = len(gene_name_polyA.split('_'))/3

    # loop through all polyA sites associated with the aligned gene
    for j in range(0,polyA_count):
        tag = gene_name_polyA.split('_')[j*3+0]
        gene = gene_name_polyA.split('_')[j*3+1]
        polyA = int(gene_name_polyA.split('_')[j*3+2])

        if (strand=='+' and int(read_end)>(polyA-polyA_dist)):
            near_polyA = 'YES'
        elif (strand=='-' and int(read_end)<(polyA+polyA_dist)):
            near_polyA = 'YES'   
    
    return near_polyA


def get_processed_splice_calls(df, read_overhang, max_softclip):
    
    df['near_polyA'] = df.apply(lambda row: get_MinION_spliceCalls_noPolyA(row.gene_name_polyA,row.strand,row.read_end,polyA_dist), axis=1)
    df_nopolyA = df[df['near_polyA']=='NO'].reset_index(drop=True)   
    df_capable = df_nopolyA[(df_nopolyA['read_length'].astype(int)-read_overhang)>df_nopolyA['dist_from_3SS'].astype(int)].reset_index(drop=True)
    df_softclip = df_capable[df_capable['end_clippling'].astype(int)<max_softclip].reset_index(drop=True)   
    
    return df_softclip


def get_splice_df(bamFile, spliceSites, min_overlap, read_overhang, max_softclip):
    
    # get reads that span 3' splice sites and convert to a dataframe
    bedFile = bamFile.bam_to_bed(cigar=True, tag='NM') # convert bam file to bed file, keep cigar string and NM (edit distance) tag

    intersect = bedFile.intersect(spliceSites, wo=True, s=True) # intersect reads from bam file with 3' splice site coordinates, ensure strandedness
    splicingReads = intersect.to_dataframe(names=['chrom', 'start_aln', 'end_aln', 'name_aln', 'qual_aln', \
                                           'strand_aln', 'cigar_aln', 'chr_3SS', 'start_3SS', \
                                           'end_3SS', 'name_gene_polyA', 'pos_5SS', 'strand_gene', 'count'], \
                               dtype={"chrom": str, "start_aln": int, "end_aln": int, \
                                     "name_aln": str, "qual_aln": int, "strand_aln": str, \
                                     "cigar_aln": str, "chr_3SS": str, "start_3SS": int, \
                                     "end_3SS": int, "name_gene_polyA": str, \
                                     "pos_5SS": int,"strand_gene": str, "count": int}) # convert to a dataframe
    
    splicingCalls = get_MinION_spliceCalls(splicingReads, min_overlap) #determine splicing status and distance
    splice_df = get_processed_splice_calls(splicingCalls, read_overhang, max_softclip) #process files (e.g. not near polyA, splicing capable, and minimal softclip)

    return splice_df


def get_discarded_splice_df(bamFile, discarded_reads, spliceSites, min_overlap, read_overhang, max_softclip):
    
    # get reads that span 3' splice sites and convert to a dataframe
    bedFile = bamFile.bam_to_bed(cigar=True, tag='NM') # convert bam file to bed file, keep cigar string and NM (edit distance) tag
    filtered_reads_bedtool = bedFile.filter(lambda b: b.name not in discarded_reads)
    filtered_reads_bedtool = filtered_reads_bedtool.saveas()

    intersect = filtered_reads_bedtool.intersect(spliceSites, wo=True, s=True) # intersect reads from bam file with 3' splice site coordinates, ensure strandedness
    splicingReads = intersect.to_dataframe(names=['chrom', 'start_aln', 'end_aln', 'name_aln', 'qual_aln', \
                                           'strand_aln', 'cigar_aln', 'chr_3SS', 'start_3SS', \
                                           'end_3SS', 'name_gene_polyA', 'pos_5SS', 'strand_gene', 'count'], \
                               dtype={"chrom": str, "start_aln": int, "end_aln": int, \
                                     "name_aln": str, "qual_aln": int, "strand_aln": str, \
                                     "cigar_aln": str, "chr_3SS": str, "start_3SS": int, \
                                     "end_3SS": int, "name_gene_polyA": str, \
                                     "pos_5SS": int,"strand_gene": str, "count": int}) # convert to a dataframe
    
    splicingCalls = get_MinION_spliceCalls(splicingReads, min_overlap) #determine splicing status and distance
    splice_df = get_processed_splice_calls(splicingCalls, read_overhang, max_softclip) #process files (e.g. not near polyA, splicing capable, and minimal softclip)

    return splice_df


In [13]:
# set all variables for analysis
# set variables for analysis

# discarded reads
polyA_window = 50           # set window for polyA measurement from gtf file
ss_window_upstream = 50      # set window for splice sites measurement from gtf file (upstream of 5'SS)
ss_window_downstream = 10    # set window for splice sites measurement from gtf file (downstream of 5'SS)
RNAPET_window = 50          # set window for transcript ends from RNA-PET data

# splicing dataframe
min_overlap = 25    # overlap required to call splicing

# splicing dataframe
polyA_dist = 150    # distance from polyA site to designate 3' end within gene
read_overhang = 150    # distance that read has to overlap upstream of 3' SS
max_softclip = 150    # maximum softclipping allowed at the start of the read


In [14]:
# upload files for processing (K562 cells)

# upload hg38 annotation files for analysis
hg38_df = pd.read_table('/path/to/annotation_files/NCBI_RefSeq_hg38_merge_parsed_sortByNameCoord.bed',header=None)
hg38_df.columns = ['chrom','start', 'end','gene','feature','strand']

# import RNAPET bed files with gene starts and ends
K562_RNAPET_cyt = pd.read_table('/path/to/annotation_files/K562_ENCODE_RNAPET_cytosol_hg38_CrossMap.bed',header=None)
K562_RNAPET_cyt.columns = ['chrom','start','end','ID','score','strand','start2','end2','zero','number','size','loc']
K562_RNAPET_chr = pd.read_table('/path/to/annotation_files/K562_ENCODE_RNAPET_chromatin_hg38_CrossMap.bed',header=None)
K562_RNAPET_chr.columns = ['chrom','start','end','ID','score','strand','start2','end2','zero','number','size','loc']

# merge cytoplasm and chromatin RNAPET bed files
K562_RNAPET_all = pd.concat([K562_RNAPET_cyt,K562_RNAPET_chr])

# install splice sites from K562 intron files
# NOTE: the intron stringency annotation files were made using the script intronCoord_stringency_from_RNAseq.py
# and the thresholds indicated in the Methods. These files can be can be made for any cell line of interest 
# using a short-read Illumina RNA-seq BAM file and the script intronCoord_stringency_from_RNAseq.py
K562_medium_intronFile = open('/path/to/annotation_files/K562_intronCoords_mediumStringency.txt') # get intron coordinates
K562_medium_spliceSites = human_spliceSites(K562_medium_intronFile) # get splice sites

# K562 -polyA dataset for removing unannotated poly(A) sites
no_tail_bamFile = pybedtools.BedTool('/path/to/K562_4sUchr_ONT_no_tailing_hg38_minimap2_uniq_sort.bam')

# upload alignment files for processing - K562 cells poly(A)
K562_1_bamFile = pybedtools.BedTool('/path/to/K562_4sUchr_ONT_1_hg38_minimap2_uniq_sort.bam')
K562_2_bamFile = pybedtools.BedTool('/path/to/K562_4sUchr_ONT_2_hg38_minimap2_uniq_sort.bam')
K562_3_bamFile = pybedtools.BedTool('/path/to/K562_4sUchr_ONT_3_hg38_minimap2_uniq_sort.bam')

# upload alignment files for processing - K562 cells poly(A) DMSO or PlaB
K562_DMSO_1_bamFile = pybedtools.BedTool('/path/to/K562_ONT_DMSO_1_hg38_minimap2_uniq_sort.bam')
K562_DMSO_2_bamFile = pybedtools.BedTool('/path/to/K562_ONT_DMSO_2_hg38_minimap2_uniq_sort.bam')
K562_PlaB_1_bamFile = pybedtools.BedTool('/path/to/K562_ONT_PlaB_1_hg38_minimap2_uniq_sort.bam')
K562_PlaB_2_bamFile = pybedtools.BedTool('/path/to/K562_ONT_PlaB_2_hg38_minimap2_uniq_sort.bam')

# upload alignment files for processing - K562 cells poly(I)
K562_4_bamFile = pybedtools.BedTool('/path/to/K562_4sUchr_ONT_4_hg38_minimap2_uniq_sort.bam')
K562_5a_bamFile = pybedtools.BedTool('/path/to/K562_4sUchr_ONT_5a_hg38_minimap2_uniq_sort.bam')
K562_5b_bamFile = pybedtools.BedTool('/path/to/K562_4sUchr_ONT_5b_hg38_minimap2_uniq_sort.bam')


In [15]:
# collect reads that will be discarded from distance spliced analysis (K562 cells)

# create a bed file with all regions that may not be representative of a transcription position
K562_discarded_features = get_discarded_features_wiRNAPET_wiNoPolyA_bedtool(hg38_df, polyA_window, ss_window_upstream, ss_window_downstream, K562_RNAPET_all, RNAPET_window, no_tail_bamFile)

# remove reads that have 3' ends within the bed file for discarded regions
K562_1_discarded_reads = get_discarded_reads(K562_1_bamFile,K562_discarded_features)
K562_2_discarded_reads = get_discarded_reads(K562_2_bamFile,K562_discarded_features)
K562_3_discarded_reads = get_discarded_reads(K562_3_bamFile,K562_discarded_features)
K562_4_discarded_reads = get_discarded_reads(K562_4_bamFile,K562_discarded_features)
K562_5a_discarded_reads = get_discarded_reads(K562_5a_bamFile,K562_discarded_features)
K562_5b_discarded_reads = get_discarded_reads(K562_5b_bamFile,K562_discarded_features)
K562_DMSO_1_discarded_reads = get_discarded_reads(K562_DMSO_1_bamFile,K562_discarded_features)
K562_DMSO_2_discarded_reads = get_discarded_reads(K562_DMSO_2_bamFile,K562_discarded_features)
K562_PlaB_1_discarded_reads = get_discarded_reads(K562_PlaB_1_bamFile,K562_discarded_features)
K562_PlaB_2_discarded_reads = get_discarded_reads(K562_PlaB_2_bamFile,K562_discarded_features)


In [16]:
# get splice dataframes for plotting (after discarding unwanted reads) (K562 cells)
K562_1_splice_df = get_discarded_splice_df(K562_1_bamFile, K562_1_discarded_reads, K562_medium_spliceSites, min_overlap, read_overhang, max_softclip)
K562_2_splice_df = get_discarded_splice_df(K562_2_bamFile, K562_2_discarded_reads, K562_medium_spliceSites, min_overlap, read_overhang, max_softclip)
K562_3_splice_df = get_discarded_splice_df(K562_3_bamFile, K562_3_discarded_reads, K562_medium_spliceSites, min_overlap, read_overhang, max_softclip)
K562_4_splice_df = get_discarded_splice_df(K562_4_bamFile, K562_4_discarded_reads, K562_medium_spliceSites, min_overlap, read_overhang, max_softclip)
K562_5a_splice_df = get_discarded_splice_df(K562_5a_bamFile, K562_5a_discarded_reads, K562_medium_spliceSites, min_overlap, read_overhang, max_softclip)
K562_5b_splice_df = get_discarded_splice_df(K562_5b_bamFile, K562_5b_discarded_reads, K562_medium_spliceSites, min_overlap, read_overhang, max_softclip)
K562_DMSO_1_splice_df = get_discarded_splice_df(K562_DMSO_1_bamFile, K562_DMSO_1_discarded_reads, K562_medium_spliceSites, min_overlap, read_overhang, max_softclip)
K562_DMSO_2_splice_df = get_discarded_splice_df(K562_DMSO_2_bamFile, K562_DMSO_2_discarded_reads, K562_medium_spliceSites, min_overlap, read_overhang, max_softclip)
K562_PlaB_1_splice_df = get_discarded_splice_df(K562_PlaB_1_bamFile, K562_PlaB_1_discarded_reads, K562_medium_spliceSites, min_overlap, read_overhang, max_softclip)
K562_PlaB_2_splice_df = get_discarded_splice_df(K562_PlaB_2_bamFile, K562_PlaB_2_discarded_reads, K562_medium_spliceSites, min_overlap, read_overhang, max_softclip)


In [17]:
# save splice dataframes to file (to use again later for plotting) (K562 cells)
K562_1_splice_df.to_csv('/path/to/K562_1_hg38_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
K562_2_splice_df.to_csv('/path/to/K562_2_hg38_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
K562_3_splice_df.to_csv('/path/to/K562_3_hg38_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
K562_4_splice_df.to_csv('/path/to/K562_4_hg38_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
K562_5a_splice_df.to_csv('/path/to/K562_5a_hg38_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
K562_5b_splice_df.to_csv('/path/to/K562_5b_hg38_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
K562_DMSO_1_splice_df.to_csv('/path/to/K562_DMSO_1_hg38_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
K562_DMSO_2_splice_df.to_csv('/path/to/K562_DMSO_2_hg38_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
K562_PlaB_1_splice_df.to_csv('/path/to/K562_PlaB_1_hg38_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
K562_PlaB_2_splice_df.to_csv('/path/to/K562_PlaB_2_hg38_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)


In [ ]:
# Lymphoblast samples

In [23]:
# install splice sites from BL1184 intron files
GM12878_medium_intronFile = open('/path/to/annotation_files/GM12878_ENCSR000AEE_intronCoords_mediumStringency.txt') # get intron coordinates
GM12878_medium_spliceSites = human_spliceSites(GM12878_medium_intronFile) # get splice sites

# upload alignment files for processing - K562 cells poly(I)
BL1184_1_bamFile = pybedtools.BedTool('/path/to/BL1184_4sUchr_ONT_1_hg38_minimap2_uniq_sort.bam')
BL1184_2_bamFile = pybedtools.BedTool('/path/to/BL1184_4sUchr_ONT_2_hg38_minimap2_uniq_sort.bam')

# remove reads that have 3' ends within the bed file for discarded regions
BL1184_1_discarded_reads = get_discarded_reads(BL1184_1_bamFile,K562_discarded_features)
BL1184_2_discarded_reads = get_discarded_reads(BL1184_2_bamFile,K562_discarded_features)

# get splice dataframes for plotting (after discarding unwanted reads) (K562 cells)
BL1184_1_splice_df = get_discarded_splice_df(BL1184_1_bamFile, BL1184_1_discarded_reads, GM12878_medium_spliceSites, min_overlap, read_overhang, max_softclip)
BL1184_2_splice_df = get_discarded_splice_df(BL1184_2_bamFile, BL1184_2_discarded_reads, GM12878_medium_spliceSites, min_overlap, read_overhang, max_softclip)

# save splice dataframes to file (to use again later for plotting) (K562 cells)
BL1184_1_splice_df.to_csv('/path/to/BL1184_1_hg38_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
BL1184_2_splice_df.to_csv('/path/to/BL1184_2_hg38_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)


In [ ]:
# Drosophila samples

In [18]:
# prepare a bedtool file features for intersecting with read ends (S2)

# upload dm6 annotation files for analysis
dm6_df = pd.read_table('/path/to/annotation_files/dm6_RefSeq_merge_parsed_sortByNameCoord.bed',header=None)
dm6_df.columns = ['chrom','start', 'end','gene','feature','strand']
dm6_df['chrom'] = 'chr'+dm6_df['chrom']

# install splice sites from K562 intron files
S2_medium_intronFile = open('/path/to/annotation_files/S2_intronCoords_mediumStringency.txt') # get intron coordinates
S2_medium_spliceSites = drosophila_spliceSites(S2_medium_intronFile) # get splice sites

# upload alignment files for processing (S2 cells)
S2_1a_bamFile = pybedtools.BedTool('/path/to/S2_4sUchr_ONT_1a_dm6_minimap2_uniq_sort.bam')
S2_1b_bamFile = pybedtools.BedTool('/path/to/S2_4sUchr_ONT_1b_dm6_minimap2_uniq_sort.bam')
S2_2_bamFile = pybedtools.BedTool('/path/to/S2_4sUchr_ONT_2_dm6_minimap2_uniq_sort.bam')
S2_3_bamFile = pybedtools.BedTool('/path/to/S2_4sUchr_ONT_3_dm6_minimap2_uniq_sort.bam')
S2_DMSO_1_bamFile = pybedtools.BedTool('/path/to/S2_ONT_DMSO_1_dm6_minimap2_uniq_sort.bam')
S2_DMSO_2_bamFile = pybedtools.BedTool('/path/to/S2_ONT_DMSO_1_dm6_minimap2_uniq_sort.bam')
S2_PlaB_1a_bamFile = pybedtools.BedTool('/path/to/S2_ONT_PlaB_1a_dm6_minimap2_uniq_sort.bam')
S2_PlaB_1b_bamFile = pybedtools.BedTool('/path/to/S2_ONT_PlaB_1b_dm6_minimap2_uniq_sort.bam')
S2_PlaB_2_bamFile = pybedtools.BedTool('/path/to/S2_ONT_PlaB_2_dm6_minimap2_uniq_sort.bam')


In [19]:
# collect reads that will be discarded from the distance spliced analysis (S2)

# create a bed file with all regions that may not be representative of a transcription position
dm6_discarded_features = get_discarded_features_woRNAPET_bedtool(dm6_df, polyA_window, ss_window_upstream, ss_window_downstream)

# remove reads that have 3' ends within the bed file for discarded regions
S2_1a_discarded_reads = get_discarded_reads(S2_1a_bamFile,dm6_discarded_features)
S2_1b_discarded_reads = get_discarded_reads(S2_1b_bamFile,dm6_discarded_features)
S2_2_discarded_reads = get_discarded_reads(S2_2_bamFile,dm6_discarded_features)
S2_3_discarded_reads = get_discarded_reads(S2_3_bamFile,dm6_discarded_features)
S2_DMSO_1_discarded_reads = get_discarded_reads(S2_DMSO_1_bamFile,dm6_discarded_features)
S2_DMSO_2_discarded_reads = get_discarded_reads(S2_DMSO_2_bamFile,dm6_discarded_features)
S2_PlaB_1a_discarded_reads = get_discarded_reads(S2_PlaB_1a_bamFile,dm6_discarded_features)
S2_PlaB_1b_discarded_reads = get_discarded_reads(S2_PlaB_1b_bamFile,dm6_discarded_features)
S2_PlaB_2_discarded_reads = get_discarded_reads(S2_PlaB_2_bamFile,dm6_discarded_features)


In [20]:
# prepare splicing dataframe for plotting (S2)

# get dataframe that discards unwanted reads
S2_1a_splice_df = get_discarded_splice_df(S2_1a_bamFile, S2_1a_discarded_reads, S2_medium_spliceSites, min_overlap, read_overhang, max_softclip)
S2_1b_splice_df = get_discarded_splice_df(S2_1b_bamFile, S2_1b_discarded_reads, S2_medium_spliceSites, min_overlap, read_overhang, max_softclip)
S2_2_splice_df = get_discarded_splice_df(S2_2_bamFile, S2_2_discarded_reads, S2_medium_spliceSites, min_overlap, read_overhang, max_softclip)
S2_3_splice_df = get_discarded_splice_df(S2_3_bamFile, S2_3_discarded_reads, S2_medium_spliceSites, min_overlap, read_overhang, max_softclip)
S2_DMSO_1_splice_df = get_discarded_splice_df(S2_DMSO_1_bamFile, S2_DMSO_1_discarded_reads, S2_medium_spliceSites, min_overlap, read_overhang, max_softclip)
S2_DMSO_2_splice_df = get_discarded_splice_df(S2_DMSO_2_bamFile, S2_DMSO_2_discarded_reads, S2_medium_spliceSites, min_overlap, read_overhang, max_softclip)
S2_PlaB_1a_splice_df = get_discarded_splice_df(S2_PlaB_1a_bamFile, S2_PlaB_1a_discarded_reads, S2_medium_spliceSites, min_overlap, read_overhang, max_softclip)
S2_PlaB_1b_splice_df = get_discarded_splice_df(S2_PlaB_1b_bamFile, S2_PlaB_1b_discarded_reads, S2_medium_spliceSites, min_overlap, read_overhang, max_softclip)
S2_PlaB_2_splice_df = get_discarded_splice_df(S2_PlaB_2_bamFile, S2_PlaB_2_discarded_reads, S2_medium_spliceSites, min_overlap, read_overhang, max_softclip)

# save splice dataframes to file (to use again later for plotting) (K562 cells)
S2_1a_splice_df.to_csv('/path/to/S2_1a_dm6_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
S2_1b_splice_df.to_csv('/path/to/S2_1b_dm6_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
S2_2_splice_df.to_csv('/path/to/S2_2_dm6_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
S2_3_splice_df.to_csv('/path/to/S2_3_dm6_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
S2_DMSO_1_splice_df.to_csv('/path/to/S2_DMSO_1_dm6_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
S2_DMSO_2_splice_df.to_csv('/path/to/S2_DMSO_2_dm6_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
S2_PlaB_1a_splice_df.to_csv('/path/to/S2_PlaB_1a_dm6_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
S2_PlaB_1b_splice_df.to_csv('/path/to/S2_PlaB_1b_dm6_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)
S2_PlaB_2_splice_df.to_csv('/path/to/S2_PlaB_2_dm6_medIntrons_discarded_splice_df.txt', sep='\t', index=False, header=True)